In [1]:
import xarray as xr
import pandas as pd
import os

In [2]:
shilshole_ds = xr.open_dataset('sg195_Shileshole_Shakedown_10Oct2024_timeseries.nc')
display(shilshole_ds)

<xarray.Dataset> Size: 300kB
Dimensions:                                   (gps_info: 15,
                                               sg_data_point: 1644,
                                               trajectory: 5, dive: 5)
Coordinates:
    ctd_time                                  (sg_data_point) datetime64[ns] 13kB ...
    ctd_depth                                 (sg_data_point) float32 7kB ...
    latitude                                  (sg_data_point) float32 7kB ...
    longitude                                 (sg_data_point) float32 7kB ...
  * trajectory                                (trajectory) int32 20B 1 2 3 4 5
Dimensions without coordinates: gps_info, sg_data_point, dive
Data variables: (12/66)
    gps_info_dive_number                      (gps_info) int32 60B ...
    sg_data_point_dive_number                 (sg_data_point) int32 7kB ...
    log_gps_time                              (gps_info) datetime64[ns] 120B ...
    time                                      (sg_data_point) datetime64[ns] 13kB ...
    pressure                                  (sg_data_point) float32 7kB ...
    depth                                     (sg_data_point) float32 7kB ...
    ...                                        ...
    end_longitude                             (dive) float32 20B ...
    depth_avg_curr_east                       (dive) float32 20B ...
    depth_avg_curr_north                      (dive) float32 20B ...
    depth_avg_curr_qc                         (dive) |S1 5B ...
    latlong_qc                                (dive) |S1 5B ...
    glider                                    |S12 12B ...
Attributes: (12/47)
    project:                         Shileshole Shakedown 10Oct2024
    title:                           Physical, chemical, and biological data ...
    summary:                         SG195 Shileshole Shakedown 10Oct2024
    source:                          Seaglider SG195
    references:                      http://data.nodc.noaa.gov/accession/0092291
    processing_level:                1.12
    ...                              ...
    date_modified:                   2024-10-11T21:10:17Z
    uuid:                            f19793b8-881a-11ef-91ac-91df858c20f4
    base_station_version:            3.0.2
    base_station_micro_version:      0
    quality_control_version:         1.12
    Conventions:                     CF-1.6

In [3]:
#correct time
#time = shilshole_ds['time'][0].values

adjusted_time = pd.to_datetime(shilshole_ds['time'].values) + pd.DateOffset(years=19, months=7, days=15)
shilshole_ds = shilshole_ds.assign_coords(time=('sg_data_point', adjusted_time))

#check time
time = shilshole_ds['time'][0].values

print(time)

2024-10-10T16:49:55.178000000


In [4]:
#Align time with sg_data_point and apply offset
adjusted_time = pd.to_datetime(shilshole_ds['time'].values) + pd.DateOffset(years=19, months=7, days=13)

shilshole_ds = shilshole_ds.assign_coords(time=('sg_data_point', adjusted_time))

shilshole_ds['PAR_470nm'] = shilshole_ds['eng_wlbb2fl_sig470nm']
shilshole_ds['particle_concentration_700nm'] = shilshole_ds['eng_wlbb2fl_sig700nm']
shilshole_ds['chlorophyll_695nm'] = shilshole_ds['eng_wlbb2fl_sig695nm']
shilshole_ds['dive_number'] = shilshole_ds['sg_data_point_dive_number']

# add metadata
shilshole_ds['PAR_470nm'].attrs['pre_cleaning_name'] = 'eng_wlbb2fl_sig470nm'
shilshole_ds['particle_concentration_700nm'].attrs['pre_cleaning_name'] = 'eng_wlbb2fl_sig700nm'
shilshole_ds['chlorophyll_695nm'].attrs['pre_cleaning_name'] = 'eng_wlbb2fl_sig695nm'
shilshole_ds['dive_number'] .attrs['pre_cleaning_name'] = ['sg_data_point_dive_number']


#Select the relevant variables
new_shilshole_ds = shilshole_ds[['time', 'depth',  'latitude','longitude','temperature', 'salinity', 'dissolved_oxygen_sat', 'PAR_470nm', 'particle_concentration_700nm', 'chlorophyll_695nm', 'dive_number']] 

#Convert to DataFrame and save
new_shilshole_ds.to_dataframe().reset_index().to_csv('sg195_october_2024_shilshole_cleaned.csv', index=False)
new_shilshole_ds.to_netcdf('sg195_october_2024_shilshole_cleaned.nc')

In [5]:
shilshole_ds = xr.open_dataset('sg195_Shileshole_Shakedown_10Oct2024_timeseries.nc')

#Apply time apply offset (if needed)
adjusted_time = pd.to_datetime(shilshole_ds['time'].values) + pd.DateOffset(years=19, months=7, days=15)

shilshole_ds['U_DAC'] = shilshole_ds['depth_avg_curr_east']
shilshole_ds['V_DAC'] = shilshole_ds['depth_avg_curr_north']

# add metadata
shilshole_ds['U_DAC'].attrs['pre_cleaning_name'] = 'depth_avg_curr_east'
shilshole_ds['V_DAC'].attrs['pre_cleaning_name'] = 'depth_avg_curr_north'

#Select the relevant variables
new_shilshole_ds = shilshole_ds[['U_DAC', 'V_DAC', 'start_time', 'end_time', 'start_latitude', 'end_latitude', 'start_longitude', 'end_longitude']]
display(new_shilshole_ds)

#Convert to DataFrame and save
new_shilshole_ds.to_dataframe().reset_index().to_csv('sg195_october_2024_shilshole_DAC_cleaned.csv', index=False)
new_shilshole_ds.to_netcdf('sg195_october_2024_shilshole_DAC_cleaned.nc')


<xarray.Dataset> Size: 200B
Dimensions:          (dive: 5)
Dimensions without coordinates: dive
Data variables:
    U_DAC            (dive) float32 20B ...
    V_DAC            (dive) float32 20B ...
    start_time       (dive) datetime64[ns] 40B ...
    end_time         (dive) datetime64[ns] 40B ...
    start_latitude   (dive) float32 20B ...
    end_latitude     (dive) float32 20B ...
    start_longitude  (dive) float32 20B ...
    end_longitude    (dive) float32 20B ...
Attributes: (12/47)
    project:                         Shileshole Shakedown 10Oct2024
    title:                           Physical, chemical, and biological data ...
    summary:                         SG195 Shileshole Shakedown 10Oct2024
    source:                          Seaglider SG195
    references:                      http://data.nodc.noaa.gov/accession/0092291
    processing_level:                1.12
    ...                              ...
    date_modified:                   2024-10-11T21:10:17Z
    uuid:                            f19793b8-881a-11ef-91ac-91df858c20f4
    base_station_version:            3.0.2
    base_station_micro_version:      0
    quality_control_version:         1.12
    Conventions:                     CF-1.6